In [ ]:
import json

import pandas as pd
from rightmove_webscraper import RightmoveData

In [20]:
url = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

rm = RightmoveData(url)

In [21]:
rm.get_results.head()

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,225000.0,3 bedroom town house for sale,"Charolais Crescent, Lightwood, ST3",http://www.rightmove.co.uk/properties/12611226...,http://www.rightmove.co.uk/estate-agents/agent...,ST3,NaN,3.0,2022-08-18 15:56:55.553800
1,105000.0,2 bedroom terraced house for sale,"Leek Road, Stoke-on-Trent, ST1",http://www.rightmove.co.uk/properties/12190782...,http://www.rightmove.co.uk/estate-agents/agent...,ST1,NaN,2.0,2022-08-18 15:56:55.553800
2,400000.0,5 bedroom detached house for sale,"Harold Hines Way, Stoke-on-Trent, ST4",http://www.rightmove.co.uk/properties/12616823...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,NaN,5.0,2022-08-18 15:56:55.553800
3,190000.0,4 bedroom detached house for sale,"Upper Normacot Road, Normacot, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12616454...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,4.0,2022-08-18 15:56:55.553800
4,119950.0,2 bedroom town house for sale,"Baden Road, Smallthorne",http://www.rightmove.co.uk/properties/12616364...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-18 15:56:55.553800


In [22]:
rm.get_results

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,225000.0,3 bedroom town house for sale,"Charolais Crescent, Lightwood, ST3",http://www.rightmove.co.uk/properties/12611226...,http://www.rightmove.co.uk/estate-agents/agent...,ST3,NaN,3.0,2022-08-18 15:56:55.553800
1,105000.0,2 bedroom terraced house for sale,"Leek Road, Stoke-on-Trent, ST1",http://www.rightmove.co.uk/properties/12190782...,http://www.rightmove.co.uk/estate-agents/agent...,ST1,NaN,2.0,2022-08-18 15:56:55.553800
2,400000.0,5 bedroom detached house for sale,"Harold Hines Way, Stoke-on-Trent, ST4",http://www.rightmove.co.uk/properties/12616823...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,NaN,5.0,2022-08-18 15:56:55.553800
3,190000.0,4 bedroom detached house for sale,"Upper Normacot Road, Normacot, Stoke-On-Trent",http://www.rightmove.co.uk/properties/12616454...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,4.0,2022-08-18 15:56:55.553800
4,119950.0,2 bedroom town house for sale,"Baden Road, Smallthorne",http://www.rightmove.co.uk/properties/12616364...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-18 15:56:55.553800
...,...,...,...,...,...,...,...,...,...
480,795000.0,6 bedroom detached house for sale,"Ball Lane, Brown Edge, Staffordshire, ST6",http://www.rightmove.co.uk/properties/11429797...,http://www.rightmove.co.uk/estate-agents/agent...,ST6,NaN,6.0,2022-08-18 15:56:55.553800
481,65000.0,2 bedroom terraced house for sale,"Oldfield Street, Fenton",http://www.rightmove.co.uk/properties/11667558...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,2.0,2022-08-18 15:56:55.553800
482,250000.0,4 bedroom terraced house for sale,"Victoria Road Methodist Church, ST4 2HG",http://www.rightmove.co.uk/properties/11076557...,http://www.rightmove.co.uk/estate-agents/agent...,ST4,ST4 2HG,4.0,2022-08-18 15:56:55.553800
483,115000.0,3 bedroom semi-detached house for sale,"Brookhouse Road, Meir, Stoke-On-Trent, ST3",http://www.rightmove.co.uk/properties/12143530...,http://www.rightmove.co.uk/estate-agents/agent...,ST3,NaN,3.0,2022-08-18 15:56:55.553800


In [70]:
path = 'city_names_extended.txt'
cities = []

In [71]:
with open(path) as file:
    for line in file:
        cities.append(line.replace("\n", ""))

In [72]:
cities

['Barnsley',
 'Basildon',
 'Basingstoke',
 'Bath',
 'Bedford',
 'Birkenhead',
 'Birmingham',
 'Blackburn',
 'Blackpool',
 'Bolton',
 'Bournemouth',
 'Bracknell',
 'Bradford',
 'Brighton and Hove',
 'Bristol',
 'Burnley',
 'Burton upon Trent',
 'Bury',
 'Cambridge',
 'Cardiff',
 'Carlisle',
 'Chatham',
 'Chelmsford',
 'Cheltenham',
 'Chester',
 'Chesterfield',
 'Colchester',
 'Coventry',
 'Crawley',
 'Darlington',
 'Derby',
 'Doncaster',
 'Dudley',
 'Eastbourne',
 'Exeter',
 'Gateshead',
 'Gillingham',
 'Gloucester',
 'Grimsby',
 'Guildford',
 'Halifax',
 'Harlow',
 'Harrogate',
 'Hartlepool',
 'Hastings',
 'Hemel Hempstead',
 'High Wycombe',
 'Huddersfield',
 'Ipswich',
 'Kingston upon Hull',
 'Leeds',
 'Leicester',
 'Lincoln',
 'Liverpool',
 'London',
 'Luton',
 'Maidstone',
 'Manchester',
 'Mansfield',
 'Middlesbrough',
 'Milton Keynes',
 'Newcastle upon Tyne',
 'Newcastle-under-Lyme',
 'Newport',
 'Northampton',
 'Norwich',
 'Nottingham',
 'Nuneaton',
 'Oldham',
 'Oxford',
 'Peterbo

In [73]:
city_encodings = {}

In [74]:
import requests, json
import re
path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'

In [75]:
for city in cities:
    count = 0
    for char in city:
        if count == 2:
            path += '/'
            count = 0
        if char == "-" or char == " ":
            path += '%20'
        else:
            path += char.upper()
        count += 1

    print(path)
    res = requests.get(path).json()
    out = json.dumps(res, indent=2)
    p = re.search('REGION\^(.+?),', out)
    if p:
        found = p.group(1)
    city_encodings[city] = found.rstrip('"')
    path = 'https://www.rightmove.co.uk/typeAhead/uknostreet/'



https://www.rightmove.co.uk/typeAhead/uknostreet/BA/RN/SL/EY
https://www.rightmove.co.uk/typeAhead/uknostreet/BA/SI/LD/ON
https://www.rightmove.co.uk/typeAhead/uknostreet/BA/SI/NG/ST/OK/E
https://www.rightmove.co.uk/typeAhead/uknostreet/BA/TH
https://www.rightmove.co.uk/typeAhead/uknostreet/BE/DF/OR/D
https://www.rightmove.co.uk/typeAhead/uknostreet/BI/RK/EN/HE/AD
https://www.rightmove.co.uk/typeAhead/uknostreet/BI/RM/IN/GH/AM
https://www.rightmove.co.uk/typeAhead/uknostreet/BL/AC/KB/UR/N
https://www.rightmove.co.uk/typeAhead/uknostreet/BL/AC/KP/OO/L
https://www.rightmove.co.uk/typeAhead/uknostreet/BO/LT/ON
https://www.rightmove.co.uk/typeAhead/uknostreet/BO/UR/NE/MO/UT/H
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/AC/KN/EL/L
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/AD/FO/RD
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/IG/HT/ON/%20A/ND/%20H/OV/E
https://www.rightmove.co.uk/typeAhead/uknostreet/BR/IS/TO/L
https://www.rightmove.co.uk/typeAhead/uknostreet/BU/RN/L

In [76]:
city_encodings

{'Barnsley': '108',
 'Basildon': '115',
 'Basingstoke': '115',
 'Bath': '116',
 'Bedford': '129',
 'Birkenhead': '161',
 'Birmingham': '162',
 'Blackburn': '167',
 'Blackpool': '168',
 'Bolton': '182',
 'Bournemouth': '194',
 'Bracknell': '197',
 'Bradford': '198',
 'Brighton and Hove': '93554',
 'Bristol': '219',
 'Burnley': '252',
 'Burton upon Trent': '5221',
 'Bury': '258',
 'Cambridge': '274',
 'Cardiff': '281',
 'Carlisle': '283',
 'Chatham': '302',
 'Chelmsford': '307',
 'Cheltenham': '308',
 'Chester': '313',
 'Chesterfield': '315',
 'Colchester': '347',
 'Coventry': '368',
 'Crawley': '378',
 'Darlington': '406',
 'Derby': '418',
 'Doncaster': '430',
 'Dudley': '443',
 'Eastbourne': '470',
 'Exeter': '494',
 'Gateshead': '544',
 'Gillingham': '547',
 'Gloucester': '556',
 'Grimsby': '578',
 'Guildford': '580',
 'Halifax': '588',
 'Harlow': '596',
 'Harrogate': '598',
 'Hartlepool': '601',
 'Hastings': '94376',
 'Hemel Hempstead': '624',
 'High Wycombe': '637',
 'Huddersfield':

In [77]:
path = "https://www.rightmove.co.uk/property-for-sale/find.html?locationIdentifier=REGION%5E1271&sortType=6&propertyTypes=detached%2Csemi-detached%2Cterraced&includeSSTC=false&mustHave=&dontShow=&furnishTypes=&keywords="

In [78]:
data = []
counts = {}

for city in cities:
    encoding = city_encodings[city]
    old = path[path.index("REGION%"): path.index("&")]
    new = path.replace(old, "REGION^" + encoding)
    rm = RightmoveData(new)
    data.append(rm.get_results)
    counts[city] = len(rm.get_results)
    print(city, " ", counts[city])


Barnsley   556
Basildon   176
Basingstoke   176
Bath   235
Bedford   222
Birkenhead   135
Birmingham   1050
Blackburn   264
Blackpool   622
Bolton   701
Bournemouth   317
Bracknell   235
Bradford   718
Brighton and Hove   376
Bristol   1050
Burnley   319
Burton upon Trent   10
Bury   128
Cambridge   224
Cardiff   711
Carlisle   214
Chatham   210
Chelmsford   289
Cheltenham   268
Chester   299
Chesterfield   334
Colchester   317
Coventry   751
Crawley   222
Darlington   399
Derby   544
Doncaster   1050
Dudley   210
Eastbourne   196
Exeter   326
Gateshead   147
Gillingham   223
Gloucester   422
Grimsby   294
Guildford   171
Halifax   218
Harlow   140
Harrogate   222
Hartlepool   370
Hastings   70
Hemel Hempstead   287
High Wycombe   135
Huddersfield   530
Ipswich   354
Kingston upon Hull   176
Leeds   1050
Leicester   1050
Lincoln   331
Liverpool   1050
London   1050
Luton   504
Maidstone   432
Manchester   1050
Mansfield   205
Middlesbrough   578
Milton Keynes   541
Newcastle upon Tyne 

In [80]:
import pandas as pd
result = pd.concat(data)
result

,price,type,address,url,agent_url,postcode,full_postcode,number_bedrooms,search_date
0,140000.0,2 bedroom semi-detached house for sale,"John Street Way, Wombwell, S73 8FT - Viewing E...",http://www.rightmove.co.uk/properties/85815972...,http://www.rightmove.co.uk/estate-agents/agent...,S73,S73 8FT,2.0,2022-08-18 17:19:27.506480
1,290000.0,3 bedroom detached house for sale,"Montrose Avenue,Darton,Barnsley,S75 5LS",http://www.rightmove.co.uk/properties/12537744...,http://www.rightmove.co.uk/estate-agents/agent...,S75,S75 5LS,3.0,2022-08-18 17:19:27.506480
2,400000.0,3 bedroom detached house for sale,"Rotherham Road, Monk Bretton, S71",http://www.rightmove.co.uk/properties/12617442...,http://www.rightmove.co.uk/estate-agents/agent...,S71,NaN,3.0,2022-08-18 17:19:27.506480
3,110000.0,3 bedroom terraced house for sale,"Pontefract Road, Oakwell",http://www.rightmove.co.uk/properties/10952320...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,3.0,2022-08-18 17:19:27.506480
4,325000.0,5 bedroom semi-detached house for sale,"Park Grove, Barnsley, S70",http://www.rightmove.co.uk/properties/12616609...,http://www.rightmove.co.uk/estate-agents/agent...,S70,NaN,5.0,2022-08-18 17:19:27.506480
...,...,...,...,...,...,...,...,...,...
277,365000.0,5 bedroom semi-detached house for sale,"Cherry Wood Crescent, Fulford, York",http://www.rightmove.co.uk/properties/85711197...,http://www.rightmove.co.uk/estate-agents/agent...,NaN,NaN,5.0,2022-08-18 17:32:55.035881
278,1500000.0,5 bedroom town house for sale,"St. Martins Lane, York, YO1 6LN",http://www.rightmove.co.uk/properties/11392433...,http://www.rightmove.co.uk/estate-agents/agent...,YO1,YO1 6LN,5.0,2022-08-18 17:32:55.035881
279,1595000.0,10 bedroom semi-detached house for sale,"St. George's Place, York, YO24",http://www.rightmove.co.uk/properties/81442933...,http://www.rightmove.co.uk/estate-agents/agent...,YO24,NaN,10.0,2022-08-18 17:32:55.035881
280,1500000.0,5 bedroom house for sale,"St. Martins Lane, York, North Yorkshire, YO1",http://www.rightmove.co.uk/properties/10194097...,http://www.rightmove.co.uk/estate-agents/agent...,YO1,NaN,5.0,2022-08-18 17:32:55.035881


In [ ]:
# Decide on city count
# Limit cities with listings minimum threshold (200?)
# Use google api to get accurate lat long from the right move address (+ use postcode if they have one) https://www.codingforentrepreneurs.com/blog/python-tutorial-google-geocoding-api/
# or use worldcities.csv
# 